# IMT TAF Fiabilité : Test de charge

- **durée** 30s

In [ ]:
%matplotlib inline
%config InlineBackend.rc = {'font.size': 10, 'figure.figsize': (16.0, 5.0),'figure.dpi': 75}
import pandas as pd
from matplotlib import pyplot as plt #%matplotlib notebook

In [ ]:
d = pd.read_csv('data/stats.csv', parse_dates=['ts'], date_parser=lambda x: pd.to_datetime(x, unit='ms').tz_localize('UTC').tz_convert('Europe/Paris'),
                                names=['id','ts','path','latency','res','res_desc'])
d["grouppath"] = d.apply(lambda x: "/".join(x['path'].split("/")[:2]), axis=1)
d.head()

## Indicateurs - résumé

In [ ]:
d.describe()

In [ ]:
def histogroup(k):
    lat = d['latency'].loc[k]; ok = d['res'].loc[k]
    return "failed" if ok == "ko" else "t<20ms" if lat < 20 else "20ms<t<40ms" if lat < 40 else "40ms<t<200ms" if lat < 200 else "t>200ms"
histo = d.groupby(histogroup)['id'].count()
ordered_labels = ["failed", "t<20ms", "20ms<t<40ms", "40ms<t<200ms", "t>200ms"]; color_dict = {"t<20ms": '#4eac5b', "20ms<t<40ms": '#608ece', "40ms<t<200ms" : '#fffc54', "t>200ms" : '#ef7b36', "failed": 'r'}
histo = histo.reindex(pd.Categorical(histo.index,categories=ordered_labels,ordered=True)).sort_index()
colors = [color_dict[l] for l in histo.index]
histo.plot.bar(color=colors);

In [ ]:
histo.plot.pie(colors=colors,figsize=(5,5));

In [ ]:
def okcount(x):
    return x.value_counts().get("ok", 0)
def kocount(x):
    return x.value_counts().get("ko", 0)
def koper(x):
    return 100*(x.value_counts().get("ko", 0) / x.count())
def pct95th(x):
    print(x.values)
    print(x.count())
    print(x.quantile([1]))
    return x.quantile(0.95)
def pct99th(x):
    return x.quantile(0.99)
def reqpersec(x):
    return x.groupby(x.apply(lambda y: y.floor('s'))).count().mean()

#d.groupby(["grouppath"])
d.groupby(["grouppath"]).agg({'res': ['count',okcount,kocount,koper],'latency': ['mean','std','min','max'],'ts': [reqpersec]})
# d.groupby(["grouppath"]).agg({'res': [okcount]})


## Temps de réponse (ms)

In [ ]:
d.groupby(d['grouppath'])['latency'].plot();

In [ ]:
d.boxplot(column=['latency'], by='grouppath');